## Задача 1: Сравнение предложений

### Дан набор предложений, скопированных с Википедии. Каждое из них имеет "кошачью тему" в одном из трех смыслов:

    кошки (животные)
    UNIX-утилита cat для вывода содержимого файлов
    версии операционной системы OS X, названные в честь семейства кошачьих

### Ваша задача — найти два предложения, которые ближе всего по смыслу к расположенному в самой первой строке. В качестве меры близости по смыслу мы будем использовать косинусное расстояние.

In [1]:
import numpy as np
import scipy.spatial
import re

### 1. Считать файл с предложениями, и привести каждое к нижнему регистру с помощью строковой функции <code>lower()</code>.

In [2]:
with open("sentences1.txt") as f:
    content = f.readlines()
content = [x.strip().lower() for x in content]
sentences = content
print(len(content), "sentences in the array")
print(content)

22 sentences in the array
['in comparison to dogs, cats have not undergone major changes during the domestication process.', 'as cat simply catenates streams of bytes, it can be also used to concatenate binary files, where it will just concatenate sequence of bytes.', 'a common interactive use of cat for a single file is to output the content of a file to standard output.', 'cats can hear sounds too faint or too high in frequency for human ears, such as those made by mice and other small animals.', 'in one, people deliberately tamed cats in a process of artificial selection, as they were useful predators of vermin.', 'the domesticated cat and its closest wild ancestor are both diploid organisms that possess 38 chromosomes and roughly 20,000 genes.', 'domestic cats are similar in size to the other members of the genus felis, typically weighing between 4 and 5 kg (8.8 and 11.0 lb).', 'however, if the output is piped or redirected, cat is unnecessary.', 'cat with one named file is safer w

### 2. Произвести токенизацию, то есть разбиение текстов на слова. Для этого можно воспользоваться регулярным выражением, которое считает разделителем любой символ, не являющийся буквой: <code>re.split('[^a-z]', t)</code> Удалить пустые слова после разделения.

In [3]:
for index, item in enumerate(content):
    content[index] = [word for word in re.split('[^a-z]', item) if word]
print(content)

[['in', 'comparison', 'to', 'dogs', 'cats', 'have', 'not', 'undergone', 'major', 'changes', 'during', 'the', 'domestication', 'process'], ['as', 'cat', 'simply', 'catenates', 'streams', 'of', 'bytes', 'it', 'can', 'be', 'also', 'used', 'to', 'concatenate', 'binary', 'files', 'where', 'it', 'will', 'just', 'concatenate', 'sequence', 'of', 'bytes'], ['a', 'common', 'interactive', 'use', 'of', 'cat', 'for', 'a', 'single', 'file', 'is', 'to', 'output', 'the', 'content', 'of', 'a', 'file', 'to', 'standard', 'output'], ['cats', 'can', 'hear', 'sounds', 'too', 'faint', 'or', 'too', 'high', 'in', 'frequency', 'for', 'human', 'ears', 'such', 'as', 'those', 'made', 'by', 'mice', 'and', 'other', 'small', 'animals'], ['in', 'one', 'people', 'deliberately', 'tamed', 'cats', 'in', 'a', 'process', 'of', 'artificial', 'selection', 'as', 'they', 'were', 'useful', 'predators', 'of', 'vermin'], ['the', 'domesticated', 'cat', 'and', 'its', 'closest', 'wild', 'ancestor', 'are', 'both', 'diploid', 'organism

### 3. Составить список всех слов, встречающихся в предложениях. Сопоставить каждому слову индекс от нуля до $(d - 1)$, где $d$ — число различных слов в предложениях. Для этого  воспользоваться структурой <b>dict</b>.

In [4]:
words = list(set(sum(content, [])))

In [5]:
words = {index: word for index, word in enumerate(words)}
print(words)

{0: 'chromosomes', 1: 'over', 2: 'binary', 3: 'instead', 4: 'os', 5: 'file', 6: 'made', 7: 'since', 8: 'human', 9: 'domestic', 10: 'for', 11: 's', 12: 'catenates', 13: 'features', 14: 'computers', 15: 'safer', 16: 't', 17: 'installation', 18: 'of', 19: 'unix', 20: 'releases', 21: 'simply', 22: 'through', 23: 'one', 24: 'domesticated', 25: 'safari', 26: 'their', 27: 'bytes', 28: 'so', 29: 'just', 30: 'x', 31: 'use', 32: 'new', 33: 'cats', 34: 'streams', 35: 'basic', 36: 'be', 37: 'stdin', 38: 'delete', 39: 'without', 40: 'diploid', 41: 'content', 42: 'useful', 43: 'create', 44: 'predecessor', 45: 'roughly', 46: 'ancestor', 47: 'members', 48: 'redirection', 49: 'read', 50: 'receives', 51: 'too', 52: 'keyboards', 53: 'by', 54: 'separate', 55: 'fifth', 56: 'type', 57: 'stdout', 58: 'closest', 59: 'possess', 60: 'yosemite', 61: 'information', 62: 'kg', 63: 'right', 64: 'mid', 65: 'osx', 66: 'allows', 67: 'according', 68: 'most', 69: 'offered', 70: 'disk', 71: 'changes', 72: 'a', 73: 'releas

### 4. Создать матрицу размера $n * d$, где $n$ — число предложений. Заполнить ее: элемент с индексом $(i, j)$ в этой матрице должен быть равен количеству вхождений $j$-го слова в $i$-е предложение. Должна получиться матрица размера 22 * 254. 

### Найти косинусное расстояние от предложения в самой первой строке (In comparison to dogs, cats have not undergone...) до всех остальных с помощью функции <code>scipy.spatial.distance.cosine</code>. Какие номера у двух предложений, ближайших к нему по этому расстоянию (строки нумеруются с нуля)? Эти два числа и будут ответами на задание. Само предложение (In comparison to dogs, cats have not undergone... ) имеет индекс 0. 

In [6]:
matrix = np.array([[sentence.count(words[j]) for j in words] for i, sentence in enumerate(sentences)])
print(matrix.shape)

dists = scipy.spatial.distance.cdist(matrix[0:1], matrix[1:], metric='cosine').flatten().tolist()
answer = {index + 1 : value for index, value in enumerate(dists)}
answer = dict(sorted(answer.items(), key=lambda x: x[1]))
print(answer)

(22, 254)
{6: 0.7327387580875756, 4: 0.7770887149698589, 21: 0.8250364469440586, 10: 0.8328165362273942, 12: 0.8396432548525454, 16: 0.8406361854220809, 20: 0.8427572744917122, 2: 0.8644738145642124, 13: 0.8703592552895671, 14: 0.8740118423302576, 11: 0.8804771390665607, 8: 0.884272487528431, 19: 0.8885443574849294, 3: 0.8951715163278082, 9: 0.9055088817476932, 7: 0.9258750683338899, 5: 0.9402385695332803, 15: 0.9442721787424647, 18: 0.9442721787424647, 1: 0.9527544408738466, 17: 0.956644501523794}


### 5. Записать полученные числа в файл, разделив пробелом. Файл должен состоять из одной строки, в конце которой не должно быть переноса.

In [7]:
answer_values = list(answer.keys())[0:2]
print(answer_values)

[6, 4]


In [8]:
f = open('submission-1.txt', 'w')
f.write(f"{answer_values[0]} {answer_values[1]}")
f.close()